In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [4]:
MIN_MATCH_COUNT = 10

template = cv2.imread('./c159/template.jpeg', 0)
cap = cv2.VideoCapture(0)

# 트랙바 선언
cv2.namedWindow('image')
cv2.createTrackbar('Threshold', 'image', 0, 100, lambda x: None)
cv2.setTrackbarPos('Threshold', 'image', 60)

sift = cv2.SIFT_create()

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    threshold = cv2.getTrackbarPos('Threshold','image') / 100.0

    kp1, des1 = sift.detectAndCompute(gray, None)
    kp2, des2 = sift.detectAndCompute(template, None)
    
    matches = flann.knnMatch(des1, des2, k=2)
    good = []
    
    for m,n in matches:
        if m.distance < threshold * n.distance:
            good.append(m)
            
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1, 1, 2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1, 1, 2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
        h,w = gray.shape
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0] ]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)
        gray = cv2.polylines(gray, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
    else:
        #print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))
        matchesMask = None
        
    draw_params = dict(matchColor = (0, 255, 0), singlePointColor = None, matchesMask = matchesMask, flags = 2)
    img3 = cv2.drawMatches(gray, kp1, template, kp2, good, None, **draw_params)

    # 이미지 출력
    cv2.imshow("image", img3)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

[ WARN:0] global /private/var/folders/b7/g6qfbypj0tq32j5_trjh516r0000gn/T/pip-req-build-oqad5xi3/opencv/modules/highgui/src/window.cpp (704) createTrackbar UI/Trackbar(Threshold@image): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.


KeyboardInterrupt: 

In [5]:
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1